In [1]:
import pandas as pd

In [2]:
#Import data file
data = pd.read_csv("Copy of deliveries.csv")

In [3]:
data = data[["match_id","inning","batting_team","bowling_team","total_runs","over"]]

In [4]:
data.head()

,match_id,inning,batting_team,bowling_team,total_runs,over
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,4,1
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,2,1


In [5]:
#Group data by match id and innings to create scores in each innings
innings_runs = data.groupby(["match_id","inning"],as_index=False)["total_runs"].sum()
innings_runs.head()

,match_id,inning,total_runs
0,1,1,207
1,1,2,172
2,2,1,184
3,2,2,187
4,3,1,183


In [8]:
#Find the number of overs played by each team per innings
over_innings = data.groupby(["match_id","inning"],as_index=False).max()
over_innings.drop(["total_runs"],axis=1,inplace=True)

In [10]:
#Create pandas table showing the bowling and batting teams, 
#number of overs per innnigs and total runs per innings
match_data = over_innings.merge(innings_runs)

In [11]:
match_data.head()

,match_id,inning,batting_team,bowling_team,over,total_runs
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,20,207
1,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,172
2,2,1,Mumbai Indians,Rising Pune Supergiant,20,184
3,2,2,Rising Pune Supergiant,Mumbai Indians,20,187
4,3,1,Gujarat Lions,Kolkata Knight Riders,20,183


In [12]:
#Calculate run rate in each innings
match_data["innings_rr"] = match_data["total_runs"]/match_data["over"]
match_data.head()

,match_id,inning,batting_team,bowling_team,over,total_runs,innings_rr
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,20,207,10.35
1,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,172,8.60
2,2,1,Mumbai Indians,Rising Pune Supergiant,20,184,9.20
3,2,2,Rising Pune Supergiant,Mumbai Indians,20,187,9.35
4,3,1,Gujarat Lions,Kolkata Knight Riders,20,183,9.15


To create the net run rate table, I will first make a table that contains the batting and bowling run rate for each team in each innings. 
After this, I will take away the bowling run rate from the batting run rate for each team for each match they play.

In [14]:
teams = list(data["batting_team"].unique()) 

In [15]:
dct = {}
for i in teams:
    dct['Batting_RR_%s' % i] = [0]
    dct['Bowling_RR_%s' % i] = [0]
print(dct)

{'Batting_RR_Sunrisers Hyderabad': [0], 'Bowling_RR_Sunrisers Hyderabad': [0], 'Batting_RR_Royal Challengers Bangalore': [0], 'Bowling_RR_Royal Challengers Bangalore': [0], 'Batting_RR_Mumbai Indians': [0], 'Bowling_RR_Mumbai Indians': [0], 'Batting_RR_Rising Pune Supergiant': [0], 'Bowling_RR_Rising Pune Supergiant': [0], 'Batting_RR_Gujarat Lions': [0], 'Bowling_RR_Gujarat Lions': [0], 'Batting_RR_Kolkata Knight Riders': [0], 'Bowling_RR_Kolkata Knight Riders': [0], 'Batting_RR_Kings XI Punjab': [0], 'Bowling_RR_Kings XI Punjab': [0], 'Batting_RR_Delhi Daredevils': [0], 'Bowling_RR_Delhi Daredevils': [0], 'Batting_RR_Chennai Super Kings': [0], 'Bowling_RR_Chennai Super Kings': [0], 'Batting_RR_Rajasthan Royals': [0], 'Bowling_RR_Rajasthan Royals': [0], 'Batting_RR_Deccan Chargers': [0], 'Bowling_RR_Deccan Chargers': [0], 'Batting_RR_Kochi Tuskers Kerala': [0], 'Bowling_RR_Kochi Tuskers Kerala': [0], 'Batting_RR_Pune Warriors': [0], 'Bowling_RR_Pune Warriors': [0], 'Batting_RR_Rising 

In [16]:
#Create a table with the bowling and batting run rates of each team
rr = pd.DataFrame(dct,match_data["match_id"])
rr.reset_index(inplace=True)
rr.head()

,match_id,Batting_RR_Sunrisers Hyderabad,Bowling_RR_Sunrisers Hyderabad,Batting_RR_Royal Challengers Bangalore,Bowling_RR_Royal Challengers Bangalore,Batting_RR_Mumbai Indians,Bowling_RR_Mumbai Indians,Batting_RR_Rising Pune Supergiant,Bowling_RR_Rising Pune Supergiant,Batting_RR_Gujarat Lions,Bowling_RR_Gujarat Lions,Batting_RR_Kolkata Knight Riders,Bowling_RR_Kolkata Knight Riders,Batting_RR_Kings XI Punjab,Bowling_RR_Kings XI Punjab,Batting_RR_Delhi Daredevils,Bowling_RR_Delhi Daredevils,Batting_RR_Chennai Super Kings,Bowling_RR_Chennai Super Kings,Batting_RR_Rajasthan Royals,Bowling_RR_Rajasthan Royals,Batting_RR_Deccan Chargers,Bowling_RR_Deccan Chargers,Batting_RR_Kochi Tuskers Kerala,Bowling_RR_Kochi Tuskers Kerala,Batting_RR_Pune Warriors,Bowling_RR_Pune Warriors,Batting_RR_Rising Pune Supergiants,Bowling_RR_Rising Pune Supergiants,Batting_RR_Delhi Capitals,Bowling_RR_Delhi Capitals
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
match_data = match_data.merge(rr)
match_data.drop_duplicates(inplace=True)
match_data.reset_index(inplace=True)
match_data.drop(["index"],axis=1,inplace=True)

In [18]:
#Populate math_data table with run rates for each team
batting_columns = list(rr.iloc[:,range(1,30,2)])
bowling_columns = list(rr.iloc[:,range(2,31,2)])
r=0
for i in batting_columns:
    for j in match_data.index:
        if str((match_data.iloc[j,2])) in i:
            match_data.loc[j,i] = match_data.iloc[j,6]
for i in bowling_columns:
    for j in match_data.index:
        if str((match_data.iloc[j,3])) in i:
            match_data.loc[j,i] = match_data.iloc[j,6]            

In [19]:
match_data.head()

,match_id,inning,batting_team,bowling_team,over,total_runs,innings_rr,Batting_RR_Sunrisers Hyderabad,Bowling_RR_Sunrisers Hyderabad,Batting_RR_Royal Challengers Bangalore,Bowling_RR_Royal Challengers Bangalore,Batting_RR_Mumbai Indians,Bowling_RR_Mumbai Indians,Batting_RR_Rising Pune Supergiant,Bowling_RR_Rising Pune Supergiant,Batting_RR_Gujarat Lions,Bowling_RR_Gujarat Lions,Batting_RR_Kolkata Knight Riders,Bowling_RR_Kolkata Knight Riders,Batting_RR_Kings XI Punjab,Bowling_RR_Kings XI Punjab,Batting_RR_Delhi Daredevils,Bowling_RR_Delhi Daredevils,Batting_RR_Chennai Super Kings,Bowling_RR_Chennai Super Kings,Batting_RR_Rajasthan Royals,Bowling_RR_Rajasthan Royals,Batting_RR_Deccan Chargers,Bowling_RR_Deccan Chargers,Batting_RR_Kochi Tuskers Kerala,Bowling_RR_Kochi Tuskers Kerala,Batting_RR_Pune Warriors,Bowling_RR_Pune Warriors,Batting_RR_Rising Pune Supergiants,Bowling_RR_Rising Pune Supergiants,Batting_RR_Delhi Capitals,Bowling_RR_Delhi Capitals
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,20,207,10.35,10.35,0.0,0.0,10.35,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,1,2,Royal Challengers Bangalore,Sunrisers Hyderabad,20,172,8.60,0.00,8.6,8.6,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,2,1,Mumbai Indians,Rising Pune Supergiant,20,184,9.20,0.00,0.0,0.0,0.00,9.2,0.00,0.00,9.2,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,9.2,0.0,0.0
3,2,2,Rising Pune Supergiant,Mumbai Indians,20,187,9.35,0.00,0.0,0.0,0.00,0.0,9.35,9.35,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.35,0.0,0.0,0.0
4,3,1,Gujarat Lions,Kolkata Knight Riders,20,183,9.15,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.0,9.15,0.0,0.0,9.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [20]:
match_data["batting_team"].unique()

array(['Sunrisers Hyderabad', 'Royal Challengers Bangalore',
       'Mumbai Indians', 'Rising Pune Supergiant', 'Gujarat Lions',
       'Kolkata Knight Riders', 'Kings XI Punjab', 'Delhi Daredevils',
       'Chennai Super Kings', 'Rajasthan Royals', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Rising Pune Supergiants',
       'Delhi Capitals'], dtype=object)

In [21]:
net_SRH = []
net_RCB = []
net_MI = []
net_RPS = []
net_GL = []
net_KKR = []
net_KXIP = []
net_DD= []
net_CSK = []
net_RR = []
net_DC = []
net_KTK = []
net_PW = []
net_RPS = []
net_DCa = []

grouped = match_data.groupby(["match_id"]).sum()
grouped.reset_index(inplace=True)
grouped.head()

for i in grouped.index:
    net_SRH.append(grouped.iloc[i,5] - grouped.iloc[i,6])
for i in grouped.index:
    net_RCB.append(grouped.iloc[i,7] - grouped.iloc[i,8])
for i in grouped.index:
    net_MI.append(grouped.iloc[i,9] - grouped.iloc[i,10])
for i in grouped.index:
    net_RPS.append(grouped.iloc[i,11] - grouped.iloc[i,12])
for i in grouped.index:
    net_GL.append(grouped.iloc[i,13] - grouped.iloc[i,14])
for i in grouped.index:
    net_KKR.append(grouped.iloc[i,15] - grouped.iloc[i,16])
for i in grouped.index:
    net_KXIP.append(grouped.iloc[i,17] - grouped.iloc[i,18])
for i in grouped.index:
    net_DD.append(grouped.iloc[i,19] - grouped.iloc[i,20])
for i in grouped.index:
    net_CSK.append(grouped.iloc[i,21] - grouped.iloc[i,22])
for i in grouped.index:
    net_RR.append(grouped.iloc[i,23] - grouped.iloc[i,24])
for i in grouped.index:
    net_DC.append(grouped.iloc[i,25] - grouped.iloc[i,26])
for i in grouped.index:
    net_KTK.append(grouped.iloc[i,27] - grouped.iloc[i,28])
for i in grouped.index:
    net_PW.append(grouped.iloc[i,29] - grouped.iloc[i,30])
for i in grouped.index:
    net_RPS.append(grouped.iloc[i,31] - grouped.iloc[i,32])
for i in grouped.index:
    net_DCa.append(grouped.iloc[i,33] - grouped.iloc[i,34])

In [22]:
net_RR = [net_SRH,net_RCB,net_MI,net_RPS,net_GL,net_KKR,net_KXIP,net_DD,net_CSK,net_RR,net_DC,net_KTK,net_PW,net_RPS,net_DCa]
df = pd.DataFrame(net_RR).transpose()
df.columns=["net_SRH","net_RCB","net_MI","net_RPS","net_GL","net_KKR","net_KXIP","net_DD","net_CSK","net_RR","net_DC","net_KTK","net_PW","net_RPS","net_DCa"]
df["match_id"] = grouped["match_id"]
df.reset_index(inplace=True)

In [23]:
#Table with net run rates for each team
df.head()

,index,net_SRH,net_RCB,net_MI,net_RPS,net_GL,net_KKR,net_KXIP,net_DD,net_CSK,net_RR,net_DC,net_KTK,net_PW,net_RPS,net_DCa,match_id
0,0,1.75,-1.75,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0
1,1,0.00,0.00,-0.15,0.150000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.150000,0.0,2.0
2,2,0.00,0.00,0.00,0.000000,-3.116667,3.116667,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,3.0
3,3,0.00,0.00,0.00,-0.481579,0.000000,0.000000,0.481579,0.00,0.0,0.0,0.0,0.0,0.0,-0.481579,0.0,4.0
4,4,0.00,0.75,0.00,0.000000,0.000000,0.000000,0.000000,-0.75,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,5.0


In [ ]:
final_netRR = grouped.merge(df)
final_netRR.drop(["inning","over","total_runs","innings_rr"],axis=1,inplace=True)

In [ ]:
matches = pd.read_csv("Matches.csv")
matches = matches[["id","season","team1","team2","winner"]]

In [ ]:
#final = matches.merge(final_netRR)
matches.rename(columns={"id":"match_id"},inplace=True)

In [ ]:
final = matches.merge(final_netRR)
import numpy as np

In [ ]:
final.fillna(0)
final.replace({'0':np.nan, 0:np.nan},inplace=True)

In [ ]:
dct= {}
for index in range(2008,2020,1):
    dct['final_%s' % index] = final.loc[final["season"]==index]

In [ ]:
final_2008 = dct["final_2008"]
final_2009 = dct["final_2009"]
final_2010 = dct["final_2010"]
final_2011 = dct["final_2011"]
final_2012 = dct["final_2012"]
final_2013 = dct["final_2013"]
final_2014 = dct["final_2014"] 
final_2015 = dct["final_2015"]
final_2016 = dct["final_2016"]
final_2017 = dct["final_2017"]
final_2018 = dct["final_2018"]
final_2019 = dct["final_2019"]

In [ ]:
final_2008.iloc[:,5:] = final_2008.iloc[:,5:].cumsum()
final_2009.iloc[:,5:] = final_2009.iloc[:,5:].cumsum()
final_2010.iloc[:,5:] = final_2010.iloc[:,5:].cumsum()
final_2011.iloc[:,5:] = final_2011.iloc[:,5:].cumsum()
final_2012.iloc[:,5:] = final_2012.iloc[:,5:].cumsum()
final_2013.iloc[:,5:] = final_2013.iloc[:,5:].cumsum()
final_2014.iloc[:,5:] = final_2014.iloc[:,5:].cumsum()
final_2015.iloc[:,5:] = final_2015.iloc[:,5:].cumsum()
final_2016.iloc[:,5:] = final_2016.iloc[:,5:].cumsum()
final_2017.iloc[:,5:] = final_2017.iloc[:,5:].cumsum()
final_2018.iloc[:,5:] = final_2018.iloc[:,5:].cumsum()
final_2019.iloc[:,5:] = final_2019.iloc[:,5:].cumsum()

In [ ]:
final_2010.head()

,match_id,season,team1,team2,winner,Batting_RR_Sunrisers Hyderabad,Bowling_RR_Sunrisers Hyderabad,Batting_RR_Royal Challengers Bangalore,Bowling_RR_Royal Challengers Bangalore,Batting_RR_Mumbai Indians,...,net_KKR,net_KXIP,net_DD,net_CSK,net_RR,net_DC,net_KTK,net_PW,net_RPS,net_DCa
174,175.0,2010.0,Kolkata Knight Riders,Deccan Chargers,Kolkata Knight Riders,NaN,NaN,NaN,NaN,NaN,...,0.55,NaN,NaN,NaN,NaN,-0.55,NaN,NaN,NaN,NaN
175,176.0,2010.0,Mumbai Indians,Rajasthan Royals,Mumbai Indians,NaN,NaN,NaN,NaN,10.6,...,NaN,NaN,NaN,NaN,-0.2,NaN,NaN,NaN,NaN,NaN
176,177.0,2010.0,Kings XI Punjab,Delhi Daredevils,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.2,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,178.0,2010.0,Royal Challengers Bangalore,Kolkata Knight Riders,Kolkata Knight Riders,NaN,NaN,6.75,6.8,NaN,...,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,179.0,2010.0,Deccan Chargers,Chennai Super Kings,Deccan Chargers,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-1.55,NaN,1.00,NaN,NaN,NaN,NaN


In [ ]:
final_2008.to_csv('final_2008.csv',index=False)
final_2009.to_csv('final_2009.csv',index=False)
final_2010.to_csv('final_2010.csv',index=False)
final_2011.to_csv('final_2011.csv',index=False)
final_2012.to_csv('final_2012.csv',index=False)
final_2013.to_csv('final_2013.csv',index=False)
final_2014.to_csv('final_2014.csv',index=False)
final_2015.to_csv('final_2015.csv',index=False)
final_2016.to_csv('final_2016.csv',index=False)
final_2017.to_csv('final_2017.csv',index=False)
final_2018.to_csv('final_2018.csv',index=False)
final_2019.to_csv('final_2019.csv',index=False)